In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
data=pd.read_csv('/kaggle/input/rossmann-store-sales/train.csv')
store=pd.read_csv('/kaggle/input/rossmann-store-sales/store.csv')
test=pd.read_csv('/kaggle/input/rossmann-store-sales/test.csv')
print(data.shape)
print(store.shape)
print(test.shape)

In [ ]:
data.head()

In [ ]:
store.head()

In [ ]:
test.head()

In [ ]:
data.dtypes

In [ ]:
data.describe(include='object')

In [ ]:
data.describe()[['Sales','Customers']].loc['mean']

In [ ]:
data.describe()[['Sales','Customers']].loc['max']

In [ ]:
data.describe()[['Sales','Customers']]

In [ ]:
data.Store.nunique()
data.Store.value_counts()

In [ ]:
data.Store.value_counts().head(20).plot.bar()

In [ ]:
data.DayOfWeek.value_counts()

In [ ]:
data.Open.value_counts()

In [ ]:
data.Store.nunique()
data.Promo.value_counts()

In [ ]:
data.isnull().sum()

In [ ]:
store.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
data['Date']=pd.to_datetime(data['Date'],format='%Y-%m-%d')
store_id=data.Store.unique()[0]                 #unique()[num] num is store_id
print(store_id)
store_rows=data[data['Store']==store_id]
print(store_rows.shape)
store_rows.resample('1d',on='Date')['Sales'].sum().plot.line(figsize=(14,4))           #helps to identify missing dates and hence accordingly line plot will be plotted

In [ ]:
store_rows=data[data['Store']==store_id]
print(store_rows.shape)

In [ ]:
store_rows

In [ ]:
store_rows[store_rows['Sales']==0]              #open=0 so when store is closed sales =0

In [ ]:
test['Date']=pd.to_datetime(test['Date'],format='%Y-%m-%d')
store_test_rows=test[test['Store']==store_id]                              #same store_id taken fr test data
store_test_rows['Date'].min(),store_test_rows['Date'].max()

In [ ]:
store_test_rows['Open'].value_counts()

In [ ]:
store_rows['Sales'].plot.hist()          #here not perfectly skew and need to identify whether 0 values are needed or not

In [ ]:
# 1 should we retain row with zero sales?
# 2 since sales column is right skewed transformation?
# 3 missing values?

In [ ]:
store[store['Store']==store_id].T #here we cannont fill promo2 since year missing values with zero as it has year dd-mm-yy in it and 0 makes no sense
#for promo2since week we can feel missing values with zer

In [ ]:
store[~store['Promo2SinceYear'].isna()]
#taking rows with notnull values                     

In [ ]:
#taking first row
store[~store['Promo2SinceYear'].isna()].iloc[0]
#taking rows with notnull values                     

In [ ]:
store['Promo2SinceWeek']=store['Promo2SinceWeek'].fillna(0)
store['Promo2SinceYear']=store['Promo2SinceYear'].fillna(store['Promo2SinceYear'].mode().iloc[0])
store['PromoInterval']=store['PromoInterval'].fillna(store['PromoInterval'].mode().iloc[0])

In [ ]:

store['CompetitionDistance'] = store['CompetitionDistance'].fillna(store['CompetitionDistance'].max())                               
store['CompetitionOpenSinceMonth']=store['CompetitionOpenSinceMonth'].fillna(store['CompetitionOpenSinceMonth'].mode().iloc[0])               
store['CompetitionOpenSinceYear']=store['CompetitionOpenSinceYear'].fillna(store['CompetitionOpenSinceYear'].mode().iloc[0])
store.isna().sum()


In [ ]:
#if sequential then go for backward and forward fill method-- eg stock market analysis mon-friday sat sun no values can ill it with fridays data

In [ ]:
data_merged=data.merge(store,on='Store',how='left')
print(data.shape)
print(data_merged.shape)

In [ ]:
data_merged

In [ ]:
data_merged.isnull().sum()

In [ ]:
##Encoding
# 3 categorical column,1 datecolumn,rest are numerical
#data_merged.dtyes
data_merged['day']=data_merged['Date'].dt.day
data_merged['month']=data_merged['Date'].dt.month
data_merged['year']=data_merged['Date'].dt.year


In [ ]:
#for linear regresson and logistic regression use 1 hoc encoding
#for decisio tree we can use label encodig
data_merged.dtypes

In [ ]:
data_merged.head()

In [ ]:
#label encoding on stateholiday,store type,assortment,promo interval       #label encoding done as we use decsion tree algorithm for linear regression
data_merged['StateHoliday'].unique()
data_merged['StateHoliday']=data_merged['StateHoliday']=data_merged['StateHoliday'].map({'0':0,0:0,'a':1,'b':2,'c':3})   #coded '0',0,a,b,c 
data_merged['StateHoliday']=data_merged['StateHoliday'].astype(int)

In [ ]:
#while using decision trees standardization is not needed
#if target column required in decision tree may need to scale it

In [ ]:
data_merged['Assortment']=data_merged['Assortment'].map({'a':1,'b':2,'c':3})   #coded a,b,c
data_merged['Assortment']=data_merged['Assortment'].astype(int)


In [ ]:
data_merged['StoreType']=data_merged['StoreType'].map({'a':1,'b':2,'c':3,'d':4})
data_merged['StoreType']=data_merged['StoreType'].astype(int)

In [ ]:
map_promo={'Jan,Apr,Jul,Oct':1,'Feb,May,Aug,Nov':2,'Mar,Jun,Sept,Dec':3}
data_merged['PromoInterval']=data_merged['PromoInterval'].map(map_promo)

In [ ]:
data_merged.dtypes

In [ ]:
features=data_merged.columns.drop(['Sales','Date','Customers'])   #customers added later on for dropping not now discussed below by dropped
from sklearn.model_selection import train_test_split
train_x,validate_x,train_y,validate_y=train_test_split(data_merged[features],np.log(data_merged['Sales']+1),test_size=0.2)           #if target variable has lot of zero vvalues use log(x+1) transform
#here test_size=0.2 is for validate_x wichis like test set.

In [ ]:
train_x.shape,validate_x.shape,train_y.shape,validate_y.shape

In [ ]:
## apply decision tree algorithm
from sklearn.tree import DecisionTreeRegressor
model_dt=DecisionTreeRegressor(max_depth=11,random_state=1).fit(train_x,train_y)
validate_y_pred=model_dt.predict(validate_x)

In [ ]:
# def draw_tree(model, columns):
#     import pydotplus
#     from sklearn.externals.six import StringIO
#     from IPython.display import Image
#     import os
#     from sklearn import tree
    
#     graphviz_path = 'C:\Program Files (x86)\Graphviz2.38/bin/'
#     os.environ["PATH"] += os.pathsep + graphviz_path

#     dot_data = StringIO()
#     tree.export_graphviz(model,
#                          out_file=dot_data,
#                          feature_names=columns)
#     graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
#     return Image(graph.create_png())


In [ ]:
# !pip install pydotplus

In [ ]:
# draw_tree(model_dt,features)         #worked only for depth =3 #plotting decision tree

In [ ]:
validate_y_pred=model_dt.predict(validate_x)
from sklearn.metrics import mean_squared_error
validate_y_inv=np.exp(validate_y)-1
validate_y_pred_inv=np.exp(validate_y_pred)-1
np.sqrt(mean_squared_error(validate_y_inv,validate_y_pred_inv))

## customer column is not present in test set do we drop it or addi t in test data

In [ ]:
import matplotlib.pyplot as plt               #for checking highest important feature
plt.figure(figsize=(10,5))                    
yvalues=model_dt.feature_importances_
xvalues=features
plt.bar(xvalues,yvalues)
plt.xticks(rotation=90)


## customer column is the most important feauture so cannot drop it

## one way of checking the important features is using heatmap or the following method

In [ ]:
data_merged.corr().loc['Sales'].sort_values(ascending=False)

In [ ]:
test.head()

## adding customer column in test data

In [ ]:
stores_avg_custs=data.groupby(['Store'])[['Customers']].mean().reset_index().astype(int)     # taking mean customers in each store

In [ ]:
test_1=test.merge(stores_avg_custs,on='Store',how='left')
test.shape,test_1.shape
test_merged=test_1.merge(store,on='Store',how='left')
test_merged.isna().sum()                             #missing values appearing as merged the data with a dataframe

In [ ]:
test_merged['Open']=test_merged['Open'].fillna(1)
test_merged['Date']=pd.to_datetime(test_merged['Date'],format='%Y-%m-%d')
test_merged['day']=test_merged['Date'].dt.day
test_merged['month']=test_merged['Date'].dt.month
test_merged['year']=test_merged['Date'].dt.year


In [ ]:
test_merged.dtypes

In [ ]:
#label encoding the object types 

In [ ]:
test_merged['StateHoliday'].unique()

In [ ]:
test_merged['StateHoliday']=test_merged['StateHoliday'].map({'0':0,'a':1})   #coded a,b,c
test_merged['Assortment']=data_merged['Assortment'].astype(int)

In [ ]:
test_merged['Assortment']=data_merged['Assortment'].map({'a':1,'b':2,'c':3})   #coded a,b,c
test_merged['Assortment']=data_merged['Assortment'].astype(int)

In [ ]:
test_merged['StoreType'].unique()

In [ ]:
test_merged['StoreType']=data_merged['StoreType'].map({'a':1,'b':2,'c':3,'d':4})
test_merged['StoreType']=data_merged['StoreType'].astype(int)

In [ ]:
map_promo={'Jan,Apr,Jul,Oct':1,'Feb,May,Aug,Nov':2,'Mar,Jun,Sept,Dec':3}
test_merged['PromoInterval']=test_merged['PromoInterval'].map(map_promo)

In [ ]:
test_merged.isnull().sum()

In [ ]:
test_pred=model_dt.predict(test_merged[features])
test_pred_inv=np.exp(test_pred)-1
submission_predicted=pd.DataFrame({'ID':test['Id'],'Sales':test_pred_inv})
submission_predicted.to_csv('submission.csv',index=False)
submission_predicted.head()

In [ ]:
# def ToWeight(y):
#     w = np.zeros(y.shape, dtype=float)
#     ind = y != 0
#     w[ind] = 1./(y[ind]**2)
#     return w

# def rmspe(y, yhat):
#     w = ToWeight(y)
#     rmspe = np.sqrt(np.mean( w * (y - yhat)**2 ))
#     return rmspe
# validate_y_inv=np.exp(validate_y)-1
# validate_y_pred_inv=np.exp(validate_y_pred)-1
# rmse_val=np.sqrt(mean_squared_error(validate_y_inv,validate_y_pred_inv))
# rmspe_val=rmspe(validate_y_inv,validate_y_pred_inv)
# print(rmse_val,rmspe_val)

In [ ]:
#open==0 ? should we keepthese rows
# missing value treatment
# necoding
# hyperparameters which model?
# adding customers clumn to test data

In [ ]:
#we will try by dropping customers column
#features=data_merged.columns.drop(['Customers']) will be added above

In [ ]:
# #hyperparameter tuning
# from sklearn.model_selection import GridSearchCV
# parameters={'max_depth':list(range(5,20))}   #randomly search within depths of 5-15
# base_model=DecisionTreeRegressor()
# cv_model=GridSearchCV(base_model,param_grid=parameters,cv=5,return_train_score=True).fit(train_x,train_y)



In [ ]:
cv_model.best_params_  #do not go for the best params result as it may give wrong result as it tries to overfit

In [ ]:
pd.DataFrame(cv_model.cv_results_)

In [ ]:
# import matplotlib.pyplot as plt
# df_cv_results=pd.DataFrame(cv_model.cv_results_).sort_values(by='mean_test_score',ascending=False)
# df_cv_results.set_index('param_max_depth')['mean_test_score'].plot.line()
# df_cv_results.set_index('param_max_depth')['mean_train_score'].plot.line()
# plt.legend()

In [ ]:
# Here the point where the lines starts diverging will be best param max_depth

In [ ]:
df_cv_results[df_cv_results['param_max_depth']==11].T

In [ ]:
#rscore of the test and the train in cv is meanscore(+-)std

In [ ]:
## for max depth =11

In [ ]:
# ## apply decision tree algorithm
# from sklearn.tree import DecisionTreeRegressor
# model_dt=DecisionTreeRegressor(max_depth=11,random_state=1).fit(train_x,train_y)
# validate_y_pred=model_dt.predict(validate_x)

In [ ]:
# test_pred=model_dt.predict(test_merged[features])
# test_pred_inv=np.exp(test_pred)-1
# submission_predicted=pd.DataFrame({'ID':test['Id'],'Sales':test_pred_inv})
# submission_predicted.to_csv('submission.csv',index=False)
# submission_predicted.head()

In [ ]:
# def ToWeight(y):
#     w = np.zeros(y.shape, dtype=float)
#     ind = y != 0
#     w[ind] = 1./(y[ind]**2)
#     return w

# def rmspe(y, yhat):
#     w = ToWeight(y)
#     rmspe = np.sqrt(np.mean( w * (y - yhat)**2 ))
#     return rmspe
# validate_y_inv=np.exp(validate_y)-1
# validate_y_pred_inv=np.exp(validate_y_pred)-1
# rmse_val=np.sqrt(mean_squared_error(validate_y_inv,validate_y_pred_inv))
# # rmspe_val=rmspe(validate_y_inv,validate_y_pred_inv)
# print(rmse_val,rmspe_val)

In [ ]:
# def get_rmspe_score(model,input_values,y_actual):
#     y_predicted=model.predict(input_values)
#     y_actual=np.exp(y_actual)-1
#     y_predicted=np.exp(y_predicted)-1
#     score=rmspe(y_actual,y_predicted)
#     return score
# parameters={'max_depth':list(range(5,20))}  
# base_model=DecisionTreeRegressor()
# cv_model=GridSearchCV(base_model,param_grid=parameters,cv=5,return_train_score=True,scoring=get_rmspe_score).fit(train_x,train_y)
# pd.DataFrame(cv_model.cv_results_)[['params','mean_test_score','mean_train_score']]



In [ ]:
#missing value treatment
#encoding
#tranformation
#model fitting hyper parameter tuning
#prediction and evaluation

# Xgboost

In [ ]:
import xgboost as xgb




In [ ]:
dtrain=xgb.DMatrix(train_x[features],train_y)
dvalidate=xgb.DMatrix(validate_x[features],validate_y)
params={'max_depth':11,'eta':0.1,'objective':'reg:linear'}
model_xg=xgb.train(params,dtrain,200)
predict_y=model_xg.predict(dvalidate)

validate_y_inv=np.exp(validate_y)-1
validate_y_pred_inv=np.exp(validate_y_pred)-1
rmspe_val=rmspe(validate_y_inv,validate_y_pred_inv)
print(rmspe_val)